In [48]:
import yt_dlp
import csv
import os
import requests


def get_channel_info(channel_url):
    ydl_opts = {
        'quiet': True,
        'extract_flat': True
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(channel_url, download=False)
        return result['uploader']

def download_thumbnail(url, filepath):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filepath, 'wb') as file:
            file.write(response.content)
        return True
    else:
        print(f"Failed to download thumbnail from URL: {url}")
        return False
    
def get_top_videos(channel_url, max_results=30, thumbnails_folder='thumbnails', csv_path='videos_info.csv', custom_path=None):
    if custom_path:
        thumbnails_folder = os.path.join(custom_path, 'thumbnails')
        csv_path = os.path.join(custom_path, 'videos_info.csv')

    channel_name = get_channel_info(channel_url)

    ydl_opts = {
        'extract_flat': True,
        'quiet': True,
        'force_generic_extractor': True,
        'extractor_args': {
            'youtube': {
                'max_views': max_results
            }
        },
        'outtmpl': f'{thumbnails_folder}/%(id)s.jpg'  # Save thumbnails as video_id.jpg inside thumbnails subfolder
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(channel_url, download=True)
        
    videos = []
    video_count = 0  # Counter for the number of videos processed
    for video in result['entries']:
        if video_count >= max_results:
            break
        
        video_id = video['id']
        title = video['title']
        view_count = video['view_count']
        
        # Using the first thumbnail URL from the video entry
        thumbnail_url = video['thumbnails'][0]['url']
        thumbnail_path = os.path.join(thumbnails_folder, f"{video_id}.jpg")

        # Download thumbnail
        if not download_thumbnail(thumbnail_url, thumbnail_path):
            print(f"Thumbnail download failed for video ID: {video_id}")
            continue

        videos.append({
            'channel_name': channel_name,
            'id': video_id,
            'title': title,
            'view_count': view_count,
            'thumbnail': thumbnail_path
        })

        video_count += 1  # Increment the video counter, janky? 
    # Write to CSV file
    with open(csv_path, 'a', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['channel_name', 'id', 'title', 'view_count', 'thumbnail']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Check if the file is empty, if so, write the header, needed to appened to the csv instead of overrite
        if os.stat(csv_path).st_size == 0:
            writer.writeheader()

        for video in videos:
            writer.writerow(video)

    return videos

if __name__ == '__main__':
    # Replace 'CHANNEL_URL' with the URL of the channel you're interested in
    channel_url = 'https://www.youtube.com/@_______/videos'
    
    # Specify custom paths if needed
    custom_path = '/Users/evanschwartz/Desktop/New Data'  # Replace with your custom path if needed

    thumbnails_folder = 'thumbnails'
    csv_path = 'videos_info.csv'

    # Create thumbnails folder if it doesn't exist
    if not os.path.exists(thumbnails_folder):
        os.makedirs(thumbnails_folder)

    # Call the function to get top videos and save thumbnails and CSV
    top_videos = get_top_videos(channel_url, thumbnails_folder=thumbnails_folder, csv_path=csv_path, custom_path=custom_path)

    print("Thumbnails and CSV file have been saved to the specified locations.")

Thumbnails and CSV file have been saved to the specified locations.
